In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# import data (df = dataframe)
path = r'...'
df_dep = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'), index_col = False)
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

In [3]:
df_ords.shape

(3421083, 7)

### Missing Values (NaNs)

#### 1 - consistency checks on df_prods

In [4]:
# find missing values
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [5]:
df_prods_nan = df_prods[df_prods['product_name'].isnull() == True]

In [6]:
df_prods_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [7]:
df_prods.shape

(49693, 5)

#### Remove NaNs

###### Variant 1 (recommended)

In [8]:
# save dataframe in df_prods_clean - only the rows where 'product_name' ISN'T null
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]
# check
df_prods_clean.shape

(49677, 5)

###### Variant 2 (not recommended: overwriting)

In [9]:
# drops every row where 'product_name' is null
df_prods.dropna(subset = ['product_name'], inplace = True)

### Duplicates

In [10]:
# save all duplicates in df_dups
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [16]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [17]:
# drop duplicates
df_prods_clean = df_prods_clean.drop_duplicates()

In [18]:
df_prods_clean.shape

(49672, 5)

### Export

In [19]:
df_prods_clean.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_cleaned.csv'))

#### 2 - use .describe() on df_prods

In [20]:
df_prods_clean.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,9.993282
std,14340.705287,38.315784,5.850779,453.615536
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,99999.000000


###### Except for the maximum price, which is surprisingly high compared to the average/mean, everything looks realistic. 

#### 3 - check df_ords for mixed data

In [21]:
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

###### The dataframe doesn't contain mixed-type columns

#### 4 - No mixed-typed columns = nothing to be fixed.

#### 5 - Check for missing values

In [22]:
# find missing values
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
customer_id                    0
order_number                   0
order_day_of_week              0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

###### There are 206209 missing values in the column 'days_since_prior_order'.

#### 6 - Address missing values and use appropriate method

In [23]:
# create new datafram to not overwrite the original one
df_ords_clean = df_ords
# create new column 'new_customer' and set true when 'days_since_prior_order' is null
df_ords_clean['new_customer'] = df_ords['days_since_prior_order'].isnull() == True

###### The large number of missing values could be for the reason that a large part of the customers have ordered only once (= are one-time customers). We could create a Boolean variable: New customer True/False.

#### 7 - Look for duplicates

In [24]:
# save all duplicates in df_dups
df_dups = df_ords_clean[df_ords_clean.duplicated()]

In [25]:
df_dups

,Unnamed: 0,order_id,customer_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,new_customer


#### 9 - No duplicates found - No changes to be made

#### 10 - Export

In [26]:
df_ords_clean.shape

(3421083, 8)

In [27]:
df_ords_clean.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_cleaned.csv'))